In [5]:
import os
#from google.colab import drive
#drive.mount('/content/drive')
#os.chdir('/content/drive/MyDrive/Github/dus_mm/')
import numpy as np
import pandas as pd
import utils
import glob

from matplotlib import pyplot as plt
import seaborn as sb
os.chdir('data')

columnsX = ['freq','AX','BX','BY','CY','CX','DX','DY','AY']
columnsY = ['AX','BX','BY','CY','CX','DX','DY','AY','AQ','AL','BQ','BL','CQ','CL','DQ','DL']
columnsYout = ['AQ','AL','BQ','BL','CQ','CL','DQ','DL','X','Y','Q','L']

FileNotFoundError: [WinError 2] Не удается найти указанный файл: 'data'

FileNotFoundError: [WinError 2] Не удается найти указанный файл: 'data'

In [199]:
X = pd.DataFrame()
y = pd.DataFrame()
numb_files = len(glob.glob('data_output*'))
for i in glob.glob('data_output*'): 
    numb = i[11]
    if (numb == 'data_output.txt'):
        numb = ''
    y = pd.concat([y,pd.read_table(os.path.join('data_output%s.txt' % numb), header = 0, names = columnsY, 
                                   index_col = False, sep='\s+', engine='python', dtype ='float')], ignore_index = 1)
    X = pd.concat([X,pd.read_table(os.path.join('data_input%s.txt' % numb), header = 0, names = columnsX, 
                                   index_col = False, sep='\s+', engine='python', dtype ="float")], ignore_index = 1)
X = X.loc[:100*4+3]
y = y.loc[:100]
X = utils.init_in(X)
y = utils.decomposition(y)

In [200]:
class Layer():
    def __init__(self, neurons: int = 256, activation: str = 'linear'):
        self.neurons = neurons
        self.activation = activation

In [248]:
class NN_model():
  
    def __init__(self, input_dimension: int):
        self.sequential = [Layer(input_dimension)]
        self.theta = np.array([])
        self.loss = 'mean_squared_error'
        self.optimizer = 'grad_descent' #'adam'
        self.num_iterations = 1000
        self.beta_1 = 0.1
        self.beta_2 = 0.1
        self.epsilon = 0.01
        self.step_size = 0.005

    def add(self, layer: Layer):
        self.sequential.append(layer)
        layer_in = self.sequential[-2]
        layer_out = self.sequential[-1]
        gamma = np.sqrt(6)/np.sqrt(layer_in.neurons + layer_out.neurons)
        init_theta = np.random.rand(layer_out.neurons,layer_in.neurons + 1) * 2 * gamma - gamma
        self.theta = np.hstack([self.theta, init_theta.reshape(init_theta.size)])

    def activation(self, layer: Layer, x, diff: bool = False):
        if (layer.activation == 'relu'):
            if (diff == True):
                x[x >= 0] = 1
                x[x < 0] = 0
            else:
                x[x < 0] = 0
        elif (layer.activation == 'linear'):
            pass
        return x

    def feedforward(self, x):
        a = x
        a_out = np.array(a.reshape([a.size]))
        slice_theta1 = 0
        for i in range(1,len(self.sequential)):
            layer1 = self.sequential[i-1]
            layer2 = self.sequential[i]
            slice_theta2 = slice_theta1 + (layer1.neurons + 1) * layer2.neurons
            theta12 = self.theta[slice_theta1:slice_theta2].reshape(layer2.neurons, layer1.neurons + 1)
            z = np.dot(np.concatenate((np.ones([a.shape[0],1]),a),axis = 1), theta12.T)
            a = self.activation(layer2, z)
            a_out = np.hstack([a_out, a.reshape(a.size)])
            slice_theta1 = slice_theta2
        return a_out

    def backprop(self, a_out, y): 
        grad_out = np.array([])
        a_out = np.flip(a_out)
        y = np.flip(y)
        slice_theta1 = 0
        slice_a1 = 0
        for i in range(1,len(self.sequential)):
            layer1 = self.sequential[-i]
            layer2 = self.sequential[-(i+1)]
            slice_theta2 = slice_theta1 + layer1.neurons * (layer2.neurons+1)
            slice_a2 = slice_a1 + layer1.neurons * y.shape[0]
            slice_a3 = slice_a2 + layer2.neurons * y.shape[0]
            a1 = a_out[slice_a1:slice_a2].reshape((y.shape[0], layer1.neurons))
            a2 = a_out[slice_a2:slice_a3].reshape((y.shape[0], layer2.neurons))
            a2 = np.concatenate((np.ones([a2.shape[0],1]), a2),axis = 1)
            if (i == 1):
                delta = self.compute_delta_end(a1, y)
            else:
                delta = np.dot(delta, theta12[:,1:]) *  self.activation(layer1, a1, diff = 'True')
            grad = (1/y.shape[0]) * (np.dot(delta.T, a2))
            grad_out = np.hstack([grad.reshape(grad.size), grad_out])
            theta12 = self.theta[slice_theta1:slice_theta2].reshape(layer1.neurons, layer2.neurons + 1)
            slice_theta1 = slice_theta2
            slice_a1 = slice_a2
        return grad_out

    def compute_delta_end(self,a_end,y):
        if (self.loss == 'mean_squared_error'):
            delta = (a_end - y)
        return delta 
    
    def compute_loss(self, a_out, y):
        a_end = a_out[-(self.sequential[-1].neurons * y.shape[0]):]
        y = y.reshape(y.size)
        loss_function = np.sum(np.power(a_end - y, 2))/(2 * y.size)
        print(loss_function)   
        
    def optimize(self, x, y):
        x = np.array(x)
        y = np.array(y)
        m = np.zeros(self.theta.shape)
        v = np.zeros(self.theta.shape)
        if (self.optimizer == 'adam'):
            for t in range(1,self.num_iterations + 1):
                a_out = self.feedforward(x)
                self.compute_loss(a_out,y)
                grad = self.backprop(a_out, y)
                m = self.beta_1 * m + (1 - self.beta_1) * grad
                v = self.beta_2 * v + (1 - self.beta_2) * np.power(grad, 2)
                m_hat = m / (1 - np.power(self.beta_1, t))
                v_hat = v / (1 - np.power(self.beta_2, t))
                self.theta = self.theta - self.step_size * m_hat / (np.sqrt(v_hat) + self.epsilon)
        if (self.optimizer == 'grad_descent'):
            for t in range(1,self.num_iterations + 1):
                a_out = self.feedforward(x)
                self.compute_loss(a_out,y)
                grad = self.backprop(a_out, y)
                self.theta = self.theta -  self.step_size * grad
                    
    def __repr__(self):
        return str(len(self.sequential))


In [249]:
dus_mm = NN_model(X.shape[1])
dus_mm.add(Layer(64, 'relu'))
dus_mm.add(Layer(64, 'relu'))
dus_mm.add(Layer(64, 'relu'))
dus_mm.add(Layer(y.shape[1], 'linear'))

In [250]:
dus_mm.optimize(X,y)

46.485221726540644
46.20766051026938
45.92725997338341
45.64455481566411
45.35956141635694
45.07317178240486
44.785881138693135
44.498344133690814
44.21081576454119
43.9242070591842
43.63950200172529
43.35735097788962
43.077880232767285
42.80173860976085
42.52953373802614
42.26159636830233
41.997915670555216
41.73877095506742
41.48462029642095
41.235947204043676
40.99332591092606
40.75674659527204
40.52623136292438
40.302123427106984
40.084803624423074
39.87407577856874
39.67017543549388
39.472937049750406
39.28254532806135
39.098834644422894
38.921800395893754
38.751280105107064
38.587347349794136
38.42958404727024
38.27798130390288
38.13216502539265
37.992323112996154
37.85841491592999
37.73024071652255
37.60735422484655
37.489962066663814
37.377783370602415
37.27095389020203
37.16935931588922
37.07330000902856
36.98296386610674
36.89842618844325
36.81979964970481
36.7474254863088
36.68148394773908
36.622146045099434
36.569493168154764
36.5237211104881
36.48509603086548
36.4539114017

<ipython-input-248-cb10edf48182>:82: RuntimeWarning: overflow encountered in power
  loss_function = np.sum(np.power(a_end - y, 2))/(2 * y.size)
<ipython-input-248-cb10edf48182>:66: RuntimeWarning: overflow encountered in multiply
  delta = np.dot(delta, theta12[:,1:]) *  self.activation(layer1, a1, diff = 'True')
<ipython-input-248-cb10edf48182>:82: RuntimeWarning: overflow encountered in power
  loss_function = np.sum(np.power(a_end - y, 2))/(2 * y.size)
<ipython-input-248-cb10edf48182>:66: RuntimeWarning: overflow encountered in multiply
  delta = np.dot(delta, theta12[:,1:]) *  self.activation(layer1, a1, diff = 'True')



nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
na